In [14]:
import sys
import os
sys.path.append(os.path.dirname(os.getcwd()))
import scipy.io as sio
from scipy.misc import imread
import tensorflow as tf
import numpy as np
import pandas as pd
import scipy.io
import tqdm 
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from xgboost import XGBClassifier
import random
from numpy.random import choice
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, roc_curve
import sklearn.pipeline as pipeline
import sklearn.preprocessing as preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [15]:
data_folder = 'K:\\sdp_dataset\\GPDSSyntheticSignatures4k\\bmgnet_features'

In [16]:
USER_KERNEL='rbf'
TRAIN_ITERATIONS = 1

In [17]:
data_f = pd.read_csv(os.path.join(data_folder,'data_features.csv'))
visual_f = pd.read_csv(os.path.join(data_folder,'visual_features.csv'))

# MODEL SELECTION & TRAINING

In [18]:
dev_exp_ratio = 0.30
sorted_id_list = np.sort(data_f['user_id'].unique())
dev_exp_splitter=int(len(sorted_id_list)*dev_exp_ratio)
dev_val_user_ids = sorted_id_list[:dev_exp_splitter]
exp_user_ids = sorted_id_list[dev_exp_splitter:]

In [19]:
dev_val_user_ids.shape

(1200,)

In [20]:
exp_user_ids.shape

(2800,)

In [21]:
fakes_preds = []
gens_preds = []

for fold in np.arange(0,TRAIN_ITERATIONS):
    # assert len(dev_val_user_ids)==581
    np.random.shuffle(dev_val_user_ids)
    dev_user_ids = dev_val_user_ids[0:1000]
    validation_user_ids = dev_val_user_ids[1000:len(dev_val_user_ids)]
    train_idx, test_idx = train_test_split(np.arange(1,25), train_size=0.5, test_size=0.5)

    dev_df = data_f.loc[data_f['user_id'].isin(dev_user_ids)]
    dev_vf = visual_f.loc[dev_df.index]
    val_df = data_f.loc[data_f['user_id'].isin(validation_user_ids)]
    val_vf = visual_f.loc[val_df.index]

    dev_df_gen = dev_df.loc[dev_df['fakeness']==0]
    dev_df_fake = dev_df.loc[dev_df['fakeness']==1]
    dev_df_gen_12 = dev_df_gen.loc[dev_df_gen['sig_id'].isin(train_idx)]
    dev_df_valid_12 = dev_df_gen.loc[dev_df_gen['sig_id'].isin(test_idx)]
    
    train_idx, test_idx = train_test_split(np.arange(1,25), train_size=0.5)
    val_df_gen = val_df.loc[val_df['fakeness']==0]
    val_df_fake = val_df.loc[val_df['fakeness']==1]
    val_df_gen_12 = val_df_gen.loc[val_df_gen['sig_id'].isin(train_idx)]
    val_df_valid_gen_12 = val_df_gen.loc[val_df_gen['sig_id'].isin(test_idx)]

    for user_id in tqdm.tqdm(validation_user_ids, ascii=True):
        clf = SVC(C=1,gamma='scale',class_weight='balanced', probability=False, kernel=USER_KERNEL)
        y_train = (pd.concat([val_df_gen_12.loc[val_df_gen_12['user_id']==user_id],dev_df_gen.loc[dev_df_gen['user_id']!=user_id]]))['user_id']==user_id
        X_train = visual_f.loc[y_train.index]  
        clf.fit(X_train, y_train)
        y_valid_fakes = val_df_fake.loc[(val_df_fake['user_id']==user_id)]
        X_valid_f = visual_f.loc[y_valid_fakes.index]
        fakes_preds.append(clf.decision_function(X_valid_f))
        y_valid_gens = val_df_valid_gen_12.loc[val_df_valid_gen_12['user_id']==user_id]
        X_valid_g = visual_f.loc[y_valid_gens.index]
        gens_preds.append(clf.decision_function(X_valid_g))

100%|################################################################################| 200/200 [17:33<00:00,  5.50s/it]


# GLOBAL THRESHOLD SELECTION

In [23]:
flat_fakes_preds = np.expand_dims(np.array([item for sublist in fakes_preds for item in sublist]),axis=1)
flat_gens_preds = np.expand_dims(np.array([item for sublist in gens_preds for item in sublist]),axis=1)
all_preds = np.vstack((flat_fakes_preds,flat_gens_preds))
all_labels = np.vstack((np.zeros((flat_fakes_preds.shape[0],1)),np.ones((flat_gens_preds.shape[0],1))))

fpr,tpr,threshold = roc_curve(all_labels,all_preds)
fnr = 1 - tpr
EER = fpr[np.nanargmin(np.absolute((fnr - fpr)))]
eer_th = threshold[np.nanargmin(np.absolute((fnr - fpr)))]
print('EER_glob : ', EER*100,'\nEER_Threshold_glob : ', eer_th)
glob_th = eer_th

EER_glob :  6.283333333333334 
EER_Threshold_glob :  0.0454021045913533


In [24]:
assert len(fakes_preds)==len(gens_preds)
EER_accum=0
for idx,val in enumerate(fakes_preds):
    user_fakes_preds = np.expand_dims(np.array(fakes_preds[idx]),axis=1)
    user_gens_preds = np.expand_dims(np.array(gens_preds[idx]),axis=1)
    all_user_preds = np.vstack((user_fakes_preds,user_gens_preds))
    all_user_labels = np.vstack((np.zeros((user_fakes_preds.shape[0],1)),np.ones((user_gens_preds.shape[0],1)))) 
    fpr,tpr,threshold = roc_curve(all_user_labels,all_user_preds)
    fnr = 1 - tpr
    EER = fpr[np.nanargmin(np.absolute((fnr - fpr)))]
    EER_accum += EER
    
print('EER_user : ', (EER_accum*100)/len(fakes_preds)) 

EER_user :  3.133333333333331


In [25]:
print(glob_th)

0.0454021045913533


In [33]:
max(flat_fakes_preds)

array([1.2503353])

In [27]:
gens_preds

[array([ 0.39324418,  0.62008794,  0.8459887 ,  0.60401911,  0.62940804,
         0.48909455,  0.22454222,  0.78919984, -0.6801653 ,  0.44891571,
         0.85943519,  0.58715675]),
 array([ 0.2562077 ,  0.43089368,  0.62183303,  0.106577  ,  0.42307672,
         0.9837425 ,  0.54865739,  0.5088962 ,  0.61371398,  0.5772317 ,
         0.37057611, -0.22680001]),
 array([0.68630507, 0.8929457 , 1.00018608, 1.03078355, 0.70030848,
        1.08958523, 0.62226868, 0.85840531, 0.77128384, 0.97875009,
        0.74242577, 0.95616462]),
 array([0.98027815, 1.15112613, 0.31803908, 0.8358714 , 0.60378434,
        0.81459735, 0.91951383, 0.90246014, 0.69211986, 0.93519851,
        0.71702253, 0.99535493]),
 array([ 0.38588338,  0.19545867, -0.26308657,  0.74576413,  0.37684907,
         0.40275687,  0.35289981,  0.66835   ,  0.31457611, -0.06528466,
         1.23780327,  0.85018501]),
 array([ 0.49664602,  0.30964791,  0.73425276,  0.97868742,  0.70487358,
         0.70422506,  0.90259852, -0.0822

# TRAIN AND TEST ON THE EXPLOITATION SET

In [120]:
test_gens_preds = []
test_fakes_preds = []

train_idx, test_idx = train_test_split(np.arange(1,25), train_size=0.5)
exp_df = data_f.loc[data_f['user_id'].isin(exp_user_ids)]
exp_vf = visual_f.loc[exp_df.index]
exp_df_gen = exp_df.loc[exp_df['fakeness']==0]
exp_df_fake = exp_df.loc[exp_df['fakeness']==1]
exp_df_fake_10 = exp_df_fake.loc[exp_df_fake['sig_id'].isin(choice(np.arange(1,31),10,replace=False))]
exp_df_gen_12 = exp_df_gen.loc[exp_df_gen['sig_id'].isin(train_idx)]
exp_df_valid_gen_10 = exp_df_gen.loc[exp_df_gen['sig_id'].isin(test_idx[0:10])]
exp_df_gen_10 = exp_df_gen.loc[exp_df_gen['sig_id'].isin(train_idx[:-2])]

dev_val_df = data_f.loc[data_f['user_id'].isin(dev_val_user_ids)]
dev_val_vf = visual_f.loc[dev_val_df.index]
dev_val_df_gen = dev_val_df.loc[dev_val_df['fakeness']==0]
dev_val_df_valid_gen_14 = dev_val_df_gen.loc[dev_val_df_gen['sig_id'].isin(choice(np.arange(1,25),14,replace=False))]

c:\users\mert\anaconda3\envs\ml_gpu\lib\site-packages\sklearn\model_selection\_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [121]:
exp_df_gen_10

,path,user_id,sig_id,fakeness
17339,c-1201-06.jpg,1201,6,0
17341,c-1201-08.jpg,1201,8,0
17342,c-1201-09.jpg,1201,9,0
17346,c-1201-13.jpg,1201,13,0
17348,c-1201-15.jpg,1201,15,0
17349,c-1201-16.jpg,1201,16,0
17350,c-1201-17.jpg,1201,17,0
17352,c-1201-19.jpg,1201,19,0
17353,c-1201-20.jpg,1201,20,0
17354,c-1201-21.jpg,1201,21,0


In [122]:
for user_id in tqdm.tqdm(exp_user_ids, ascii=True):
    clf = SVC(C=1,gamma='scale',class_weight='balanced', probability=False, kernel=USER_KERNEL)
    y_train = (pd.concat([exp_df_gen_10.loc[exp_df_gen_10['user_id']==user_id],dev_val_df_valid_gen_14.loc[dev_val_df_valid_gen_14['user_id']!=user_id]]))['user_id']==user_id
    X_train = visual_f.loc[y_train.index]  
    clf.fit(X_train, y_train)
    y_valid_fakes = exp_df_fake_10.loc[(exp_df_fake_10['user_id']==user_id)]
    X_valid_f = visual_f.loc[y_valid_fakes.index]
    test_fakes_preds.append(clf.decision_function(X_valid_f))
    y_valid_gens = exp_df_valid_gen_10.loc[exp_df_valid_gen_10['user_id']==user_id]
    X_valid_g = visual_f.loc[y_valid_gens.index]
    test_gens_preds.append(clf.decision_function(X_valid_g))



  0%|                                                                                         | 0/2800 [00:00<?, ?it/s]

  0%|                                                                               | 1/2800 [00:03<2:27:40,  3.17s/it]

  0%|                                                                               | 2/2800 [00:06<2:27:14,  3.16s/it]

  0%|                                                                               | 3/2800 [00:10<2:37:38,  3.38s/it]

  0%|1                                                                              | 4/2800 [00:13<2:40:32,  3.45s/it]

  0%|1                                                                              | 5/2800 [00:16<2:25:32,  3.12s/it]

  0%|1                                                                              | 6/2800 [00:19<2:23:33,  3.08s/it]

  0%|1                                                                              | 7/2800 [00:23<2:34:21,  3.32s/it]

  0%|2                        

  5%|###6                                                                         | 134/2800 [07:10<2:28:19,  3.34s/it]

  5%|###7                                                                         | 135/2800 [07:13<2:31:22,  3.41s/it]

  5%|###7                                                                         | 136/2800 [07:17<2:32:40,  3.44s/it]

  5%|###7                                                                         | 137/2800 [07:19<2:18:29,  3.12s/it]

  5%|###7                                                                         | 138/2800 [07:23<2:33:01,  3.45s/it]

  5%|###8                                                                         | 139/2800 [07:27<2:34:20,  3.48s/it]

  5%|###8                                                                         | 140/2800 [07:30<2:24:05,  3.25s/it]

  5%|###8                                                                         | 141/2800 [07:34<2:33:25,  3.46s/it]

  5%|###9                       

 10%|#######3                                                                     | 268/2800 [14:19<2:27:36,  3.50s/it]

 10%|#######3                                                                     | 269/2800 [14:21<2:15:12,  3.21s/it]

 10%|#######4                                                                     | 270/2800 [14:25<2:19:10,  3.30s/it]

 10%|#######4                                                                     | 271/2800 [14:28<2:13:41,  3.17s/it]

 10%|#######4                                                                     | 272/2800 [14:31<2:15:36,  3.22s/it]

 10%|#######5                                                                     | 273/2800 [14:34<2:12:36,  3.15s/it]

 10%|#######5                                                                     | 274/2800 [14:38<2:18:02,  3.28s/it]

 10%|#######5                                                                     | 275/2800 [14:41<2:16:12,  3.24s/it]

 10%|#######5                   

 14%|###########                                                                  | 402/2800 [21:24<2:01:39,  3.04s/it]

 14%|###########                                                                  | 403/2800 [21:29<2:14:41,  3.37s/it]

 14%|###########1                                                                 | 404/2800 [21:32<2:14:02,  3.36s/it]

 14%|###########1                                                                 | 405/2800 [21:35<2:10:22,  3.27s/it]

 14%|###########1                                                                 | 406/2800 [21:39<2:16:22,  3.42s/it]

 15%|###########1                                                                 | 407/2800 [21:41<2:05:58,  3.16s/it]

 15%|###########2                                                                 | 408/2800 [21:44<1:56:05,  2.91s/it]

 15%|###########2                                                                 | 409/2800 [21:46<1:55:01,  2.89s/it]

 15%|###########2               

 19%|##############7                                                              | 536/2800 [29:02<2:16:00,  3.60s/it]

 19%|##############7                                                              | 537/2800 [29:06<2:13:49,  3.55s/it]

 19%|##############7                                                              | 538/2800 [29:08<2:00:18,  3.19s/it]

 19%|##############8                                                              | 539/2800 [29:11<1:59:33,  3.17s/it]

 19%|##############8                                                              | 540/2800 [29:14<2:01:49,  3.23s/it]

 19%|##############8                                                              | 541/2800 [29:20<2:23:27,  3.81s/it]

 19%|##############9                                                              | 542/2800 [29:24<2:28:13,  3.94s/it]

 19%|##############9                                                              | 543/2800 [29:27<2:25:45,  3.87s/it]

 19%|##############9            

...

 77%|##########################################################1                 | 2144/2800 [2:01:48<41:11,  3.77s/it]

 77%|##########################################################2                 | 2145/2800 [2:01:51<39:02,  3.58s/it]

 77%|##########################################################2                 | 2146/2800 [2:01:55<38:54,  3.57s/it]

 77%|##########################################################2                 | 2147/2800 [2:01:58<38:07,  3.50s/it]

 77%|##########################################################3                 | 2148/2800 [2:02:01<37:15,  3.43s/it]

 77%|##########################################################3                 | 2149/2800 [2:02:05<38:19,  3.53s/it]

 77%|##########################################################3                 | 2150/2800 [2:02:09<40:52,  3.77s/it]

 77%|##########################################################3                 | 2151/2800 [2:02:13<39:09,  3.62s/it]

 77%|###########################

 81%|#############################################################8              | 2278/2800 [2:10:09<33:36,  3.86s/it]

 81%|#############################################################8              | 2279/2800 [2:10:12<31:17,  3.60s/it]

 81%|#############################################################8              | 2280/2800 [2:10:16<33:14,  3.84s/it]

 81%|#############################################################9              | 2281/2800 [2:10:20<33:18,  3.85s/it]

 82%|#############################################################9              | 2282/2800 [2:10:24<32:16,  3.74s/it]

 82%|#############################################################9              | 2283/2800 [2:10:27<31:50,  3.69s/it]

 82%|#############################################################9              | 2284/2800 [2:10:30<29:55,  3.48s/it]

 82%|##############################################################              | 2285/2800 [2:10:34<29:51,  3.48s/it]

 82%|###########################

 86%|#################################################################4          | 2412/2800 [2:18:19<24:16,  3.75s/it]

 86%|#################################################################4          | 2413/2800 [2:18:22<23:41,  3.67s/it]

 86%|#################################################################5          | 2414/2800 [2:18:26<23:45,  3.69s/it]

 86%|#################################################################5          | 2415/2800 [2:18:29<23:15,  3.63s/it]

 86%|#################################################################5          | 2416/2800 [2:18:33<22:34,  3.53s/it]

 86%|#################################################################6          | 2417/2800 [2:18:37<24:10,  3.79s/it]

 86%|#################################################################6          | 2418/2800 [2:18:40<22:51,  3.59s/it]

 86%|#################################################################6          | 2419/2800 [2:18:44<22:05,  3.48s/it]

 86%|###########################

 91%|#####################################################################1      | 2546/2800 [2:26:40<14:50,  3.51s/it]

 91%|#####################################################################1      | 2547/2800 [2:26:44<14:57,  3.55s/it]

 91%|#####################################################################1      | 2548/2800 [2:26:47<14:46,  3.52s/it]

 91%|#####################################################################1      | 2549/2800 [2:26:51<15:14,  3.64s/it]

 91%|#####################################################################2      | 2550/2800 [2:26:55<14:58,  3.59s/it]

 91%|#####################################################################2      | 2551/2800 [2:26:58<15:17,  3.69s/it]

 91%|#####################################################################2      | 2552/2800 [2:27:02<14:58,  3.62s/it]

 91%|#####################################################################2      | 2553/2800 [2:27:05<14:13,  3.45s/it]

 91%|###########################

 96%|########################################################################7   | 2680/2800 [2:34:51<07:09,  3.58s/it]

 96%|########################################################################7   | 2681/2800 [2:34:54<06:44,  3.40s/it]

 96%|########################################################################7   | 2682/2800 [2:34:56<06:18,  3.20s/it]

 96%|########################################################################8   | 2683/2800 [2:35:01<06:46,  3.48s/it]

 96%|########################################################################8   | 2684/2800 [2:35:03<06:16,  3.25s/it]

 96%|########################################################################8   | 2685/2800 [2:35:06<05:44,  3.00s/it]

 96%|########################################################################9   | 2686/2800 [2:35:08<05:26,  2.87s/it]

 96%|########################################################################9   | 2687/2800 [2:35:11<05:16,  2.80s/it]

 96%|###########################

In [123]:
flat_test_fakes_preds = np.expand_dims(np.array([item for sublist in test_fakes_preds for item in sublist]),axis=1)
flat_test_gens_preds = np.expand_dims(np.array([item for sublist in test_gens_preds for item in sublist]),axis=1)
print("____At the EER threshold decided on the Validation set____")
print("FRR : ",(1-len(flat_test_gens_preds[flat_test_gens_preds>=glob_th])/len(flat_test_gens_preds))*100)
print("FARskilled : ",(1-len(flat_test_fakes_preds[flat_test_fakes_preds<glob_th])/len(flat_test_fakes_preds))*100)

____At the EER threshold decided on the Validation set____
FRR :  7.296428571428571
FARskilled :  6.7071428571428555


In [124]:
all_test_preds = np.vstack((flat_test_fakes_preds,flat_test_gens_preds))
all_test_labels = np.vstack((np.zeros((flat_test_fakes_preds.shape[0],1)),np.ones((flat_test_gens_preds.shape[0],1))))

fpr,tpr,threshold = roc_curve(all_test_labels,all_test_preds)
fnr = 1 - tpr
EER = fpr[np.nanargmin(np.absolute((fnr - fpr)))]
eer_th = threshold[np.nanargmin(np.absolute((fnr - fpr)))]
print('EER_glob for test set: ', EER*100,'\nEER_Threshold_glob for test set: ', eer_th)

EER_glob for test set:  6.950000000000001 
EER_Threshold_glob for test set:  0.03830680345687121


In [125]:
assert len(test_fakes_preds)==len(test_gens_preds)
EER_accum=0
for idx,val in enumerate(test_fakes_preds):
    user_test_fakes_preds = np.expand_dims(np.array(test_fakes_preds[idx]),axis=1)
    user_test_gens_preds = np.expand_dims(np.array(test_gens_preds[idx]),axis=1)
    all_user_test_preds = np.vstack((user_test_fakes_preds,user_test_gens_preds))
    all_user_test_labels = np.vstack((np.zeros((user_test_fakes_preds.shape[0],1)),np.ones((user_test_gens_preds.shape[0],1)))) 
    fpr,tpr,threshold = roc_curve(all_user_test_labels,all_user_test_preds)
    fnr = 1 - tpr
    EER = fpr[np.nanargmin(np.absolute((fnr - fpr)))]
    EER_accum += EER
    
print('EER_user for test set : ', (EER_accum*100)/len(test_fakes_preds))

EER_user for test set :  2.646428571428578
